In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import datasets, transforms

import sklearn
from sklearn.metrics import classification_report
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# seed 고정 (rand 함수 실행시 같은 결과 나올 수 있도록)
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)



# 0. Convolutional Layer



In [ ]:
# Define the convolutional layer
conv = nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding='same')

for p in conv.parameters():
    print(p)

Parameter containing:
tensor([[[[ 0.1471,  0.1597, -0.0451],
          [ 0.1768, -0.0422,  0.0388],
          [-0.0937,  0.1130,  0.1697]],

         [[-0.1412,  0.1673,  0.0360],
          [ 0.1422,  0.0261,  0.0928],
          [-0.0272,  0.1484,  0.0284]],

         [[-0.0898,  0.0491, -0.0887],
          [-0.0226, -0.0782,  0.1277],
          [-0.1519, -0.0887, -0.0543]]],


        [[[-0.1157,  0.0182, -0.1901],
          [ 0.1738, -0.1635,  0.1486],
          [ 0.0320, -0.0625,  0.1189]],

         [[ 0.0300,  0.1555,  0.0210],
          [-0.0607,  0.0517, -0.0522],
          [ 0.0810,  0.1718,  0.1112]],

         [[-0.0841,  0.1111,  0.0344],
          [ 0.0977, -0.1173, -0.1905],
          [-0.0744, -0.1476,  0.1579]]]], requires_grad=True)
Parameter containing:
tensor([0.0554, 0.0797], requires_grad=True)


In [ ]:
# Define the input tensor
input = torch.randn(10, 3, 7, 7) # Batch_size, Kernel_size, Height, Weight

# Apply the convolution
output = conv(input)

# Print the output shape
print(output.shape)

torch.Size([10, 2, 7, 7])


# 1. CNN

### Dataset Preparation

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST("data", train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=64, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST("data", train=False, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=64, shuffle=True
)


100%|██████████| 26.4M/26.4M [00:01<00:00, 13.3MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 211kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.94MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 9.81MB/s]


### CNN 선언

In [ ]:
# 모델 정의
class FashionMNISTModel(nn.Module):
    def __init__(self):
        super(FashionMNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = FashionMNISTModel()

In [ ]:
# Loss, Optimizer 선언
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training Loop
model.train()

for epoch in tqdm(range(5)):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 1, 28, 28)
        labels = labels

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch + 1, 5, i + 1, len(train_loader), loss.item()))

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch [1/5], Step [100/938], Loss: 0.4140
Epoch [1/5], Step [200/938], Loss: 0.4059
Epoch [1/5], Step [300/938], Loss: 0.3974
Epoch [1/5], Step [400/938], Loss: 0.3431
Epoch [1/5], Step [500/938], Loss: 0.4166
Epoch [1/5], Step [600/938], Loss: 0.3133
Epoch [1/5], Step [700/938], Loss: 0.3608
Epoch [1/5], Step [800/938], Loss: 0.3699
Epoch [1/5], Step [900/938], Loss: 0.3859


 20%|██        | 1/5 [00:58<03:52, 58.18s/it]

Epoch [2/5], Step [100/938], Loss: 0.3337
Epoch [2/5], Step [200/938], Loss: 0.4295
Epoch [2/5], Step [300/938], Loss: 0.2199
Epoch [2/5], Step [400/938], Loss: 0.2052
Epoch [2/5], Step [500/938], Loss: 0.2448
Epoch [2/5], Step [600/938], Loss: 0.2085
Epoch [2/5], Step [700/938], Loss: 0.2584
Epoch [2/5], Step [800/938], Loss: 0.3856
Epoch [2/5], Step [900/938], Loss: 0.1447


 40%|████      | 2/5 [01:49<02:42, 54.15s/it]

...

 60%|██████    | 3/5 [02:41<01:46, 53.14s/it]

Epoch [4/5], Step [100/938], Loss: 0.2925
Epoch [4/5], Step [200/938], Loss: 0.1267
Epoch [4/5], Step [300/938], Loss: 0.4202
Epoch [4/5], Step [400/938], Loss: 0.1392
Epoch [4/5], Step [500/938], Loss: 0.2732
Epoch [4/5], Step [600/938], Loss: 0.1337
Epoch [4/5], Step [700/938], Loss: 0.1779
Epoch [4/5], Step [800/938], Loss: 0.1821
Epoch [4/5], Step [900/938], Loss: 0.1739


 80%|████████  | 4/5 [03:31<00:51, 51.89s/it]

Epoch [5/5], Step [100/938], Loss: 0.2626
Epoch [5/5], Step [200/938], Loss: 0.0924
Epoch [5/5], Step [300/938], Loss: 0.2445
Epoch [5/5], Step [400/938], Loss: 0.1382
Epoch [5/5], Step [500/938], Loss: 0.1569
Epoch [5/5], Step [600/938], Loss: 0.1796
Epoch [5/5], Step [700/938], Loss: 0.1032
Epoch [5/5], Step [800/938], Loss: 0.0699
Epoch [5/5], Step [900/938], Loss: 0.1641


100%|██████████| 5/5 [04:23<00:00, 52.63s/it]


In [ ]:
# GPU 지정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(model.to(device))
next(model.parameters()).device # model parameter 위치 확인 (cuda or cpu)

cuda
FashionMNISTModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


device(type='cuda', index=0)

In [ ]:
# Loss, Optimizer 재 선언
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 모델 학습

In [ ]:
# Training Loop using GPU
for epoch in tqdm(range(5)):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 1, 28, 28).to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        # if (i + 1) % 100 == 0:
        #     print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
        #         epoch + 1, 5, i + 1, len(train_loader), loss.item()))

100%|██████████| 5/5 [01:16<00:00, 15.39s/it]


### 모델 평가

In [ ]:
model.eval()

y_pred, y_true = torch.zeros([64]).to(device), torch.zeros([64]).to(device)
total, correct = 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(-1, 1, 28, 28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        if (predicted.shape[0]!=64):
            predicted = torch.cat([predicted, torch.zeros(64-predicted.shape[0]).to(device)])
            labels = torch.cat([labels, torch.zeros(64-labels.shape[0]).to(device)])

        y_pred = torch.cat([y_pred, predicted])
        y_true = torch.cat([y_true, labels])

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Test Accuracy: {}%'.format((correct / total) * 100))

Test Accuracy: 91.43113057324841%


In [ ]:
print(classification_report(y_true.detach().cpu().numpy(), y_pred.detach().cpu().numpy()))

              precision    recall  f1-score   support

         0.0       0.89      0.85      0.87      1112
         1.0       0.99      0.99      0.99      1000
         2.0       0.82      0.90      0.86      1000
         3.0       0.92      0.92      0.92      1000
         4.0       0.89      0.84      0.86      1000
         5.0       0.98      0.97      0.98      1000
         6.0       0.75      0.76      0.76      1000
         7.0       0.95      0.98      0.96      1000
         8.0       0.98      0.98      0.98      1000
         9.0       0.97      0.96      0.97      1000

    accuracy                           0.91     10112
   macro avg       0.92      0.92      0.92     10112
weighted avg       0.92      0.91      0.91     10112

